# EDA Sentiment Analyses 

In [26]:
# Importing the Required Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from textblob import TextBlob
from nltk.corpus import stopwords
from collections import Counter
import warnings; warnings.simplefilter('ignore')
import nltk
import re
import string
from string import punctuation
from nltk import ngrams
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer

from sklearn.preprocessing import LabelEncoder 
import xgboost as xgb
#from lightgbm import LGBMModel,LGBMClassifier, plot_importance
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split

stop_words = set(stopwords.words('english'))
punctuation = punctuation + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&o'

ModuleNotFoundError: No module named 'catboost'

In [29]:
# ! conda config --add channels conda-forge


In [30]:
# ! conda install catboost

In [24]:
# install wordcloud package in anaconda prompt 
# ! pip install wordcloud
# install textblob package in anaconda prompt 
#! pip install textblob
# install nltk package in anaconda prompt 
#! pip install nltk 
#install xgboost package in anaconda prompt 
#! pip install xgboost 
#install lightgbm package in anaconda prompt 
#! pip install lightgbm 
#install catboost package in anaconda prompt 
#! pip install catboost
# install scipy package in anaconda prompt 
# ! pip install scipy

In [7]:
# UCI ML drug dataset

# Importing the datasets

# df_train = pd.read_csv("/kaggle/input/kuc-hackathon-winter-2018/drugsComTrain_raw.csv")
# df_test = pd.read_csv("/kaggle/input/kuc-hackathon-winter-2018/drugsComTest_raw.csv") 

# print ("The shape of the train set given is : ", df_train.shape)
# print ("The shape of the test set given is : ", df_test.shape)

# df_train.head()

In [8]:
# Data types
# df_train.dtypes

In [3]:
# Merging the test and train data 
# merge = [df_train, df_test]
# df_data = pd.concat(merge)

# print (df_data.shape)

# df_data.head(10)

In [9]:
# df_data.describe()

In [5]:
# Null values
# print ("Null values in the dataset : ", df_data.isnull().sum(axis = 0))